**Analysis owner**: Felipe Matheus <br>
**Date**: 12/10/2021 <br>
**Related Links**:

The objective of this notebook is to download porcesssed data from eda notebook and do a simple classification (identification) model.

# Index

1. Setup
2. Loading data
3. Launching H2O AutoML
4. Verifying results
5. Conclusions

# 1. Setup

In [1]:
import os
import h2o
import sys
import json
import pickle
import pandas as pd
import numpy as np

%load_ext autoreload
%autoreload 2

In [5]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.11" 2021-04-20; OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.20.04); OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.20.04, mixed mode, sharing)
  Starting server from /local/home/ff268477/projects/phd-data-wrangling/.eda-jeff-data/lib/python3.8/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpar9zraf_
  JVM stdout: /tmp/tmpar9zraf_/h2o_ff268477_started_from_python.out
  JVM stderr: /tmp/tmpar9zraf_/h2o_ff268477_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Europe/Paris
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.34.0.3
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_ff268477_abexp6
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.820 Gb
H2O_cluster_total_cores:,12
H2O_cluster_allowed_cores:,12
H2O_cluster_status:,"locked, healthy"


In [6]:
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.feature_engineering.FeatureEngineering import FeatureEngineering
from src.modeling.Modeling import Modeling

Variables

In [8]:
pd.options.display.max_rows = 6
with open('../config/variables.json') as json_file:
    var = json.load(json_file)
STRUCTURED_DATA = var.get('STRUCTURED_DATA')

Objects

In [50]:
feng = FeatureEngineering()
modeling = Modeling()

# 2. Loading Data

In [34]:
df = pd.read_pickle(STRUCTURED_DATA + "dataframe_21_1200.pkl")

# 3. Launching H2O AutoML

In [37]:
feng.label_dataframe(df, 'bd')

In [41]:
df = df.drop(columns = ["-1.0", "-2.0"]).fillna(0)

In [67]:
split_dict = modeling.spliting(df=df, split_ratio= 0.8, target_variable="has_bd")

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [66]:
aml = modeling.training(
    model_name= "bd_identification_model",
    target_variable= "has_bd",
    features=list(split_dict["X"].columns),
    train= split_dict["X"],
    test= split_dict["test"],
    maxmodels = 20,
    seed = 1
)

H2OTypeError: 'training_frame' must be a valid H2OFrame!

In [42]:
df

,1.0,10.0,100.0,101.0,102.0,103.0,104.0,105.0,106.0,107.0,...,889.0,899.0,990.0,875.0,944.0,945.0,967.0,-0.0,992.0,has_bd
eu_bd_0,0.000030,0.009258,0.003755,0.003283,0.003688,0.003548,0.003445,0.003445,0.003541,0.003799,...,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True
eu_bd_1,0.000044,0.009922,0.003445,0.003703,0.003305,0.003681,0.003578,0.003467,0.003497,0.003740,...,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,True
eu_bd_2,0.000059,0.009604,0.003548,0.003482,0.003489,0.003290,0.003460,0.003585,0.003076,0.003445,...,0.00003,0.000007,0.000007,0.0,0.0,0.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ba_co_1197,0.000000,0.008867,0.001977,0.001837,0.001571,0.001778,0.001734,0.001497,0.001571,0.001999,...,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False
ba_co_1198,0.000000,0.008579,0.001992,0.001586,0.001483,0.001770,0.001468,0.001608,0.001748,0.001866,...,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False
ba_co_1199,0.000000,0.008439,0.001815,0.001556,0.001711,0.001497,0.001520,0.001549,0.001903,0.001719,...,0.00000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,False
